# John Myles White's Brownian Motion Example

##### Dr. Hua Zhou, [huazhou@ucla.edu](mailto: huazhou@ucla.edu)
##### Department of Biostatistics, UCLA
##### Mar 20, 2017 @ Santen Pharmaceutical

This example is taken from John Myles White's talk at [Multithreaded Data event](http://multithreaded.stitchfix.com/blog/2015/03/05/john-myles-white-on-julia/). This is a naive implementation of Brownian motion in R:

In [9]:
using RCall
R"""
loop <- function (n) {
    x <- 0.0
    for (i in 1:n) {
       x <- x + rnorm(1)
    }
    return(x)
}
"""

RCall.RObject{RCall.ClosSxp}
function (n) 
{
    x <- 0
    for (i in 1:n) {
        x <- x + rnorm(1)
    }
    return(x)
}


Computing a single sample from this process for n = 10,000,000 takes around 35 seconds in `R`.

In [10]:
@elapsed R"""
loop(10000000)
"""

23.530458857

Here’s a translation of that R code into Julia:

In [11]:
function loop(n)
    x = 0.0
    for i in 1:n
        x = x + randn()
    end
    return x
end

loop (generic function with 1 method)

This implementation takes only 0.06 seconds to run - a whopping 400x faster than `R`.

In [12]:
@elapsed loop(10000000)

0.063280588

Some of the main reasons why the R interpreter executes so much more slowly than the Julia compiler include:

* Numerical values in R are subject to indirection because R almost never assumes that the type of a variable is constant throughout the body of a function.

* Scalar values in R don’t exist, which imposes additional indirection when you only want to work with a single scalar value.

* Because scalar values don’t exist in R, every single addition step in the main loop of this function has to allocate a new chunk of memory in which to store a vector.

* Because R allows function calls within a function to change the semantics of almost any construct in the language, every operation must check whether its semantics are unaltered at every iteration in the loop.

So what makes Julia so efficient?

* Julia infers the types of all variables inside the body of the loop function, conditional on knowing the types of the input arguments `n`.

* With the results of type inference in hand, Julia is able to ask LLVM to generate machine code at run-time. This code corresponds to what a simple translation of the type-annotated Julia code into C would compile into.

* Julia then executes the function body using the run-time compiled code rather than interpreting the raw source code.

In [13]:
@code_lowered loop(Int)

LambdaInfo template for loop(n) at In[11]:2
:(begin 
        nothing
        x = 0.0 # line 3:
        SSAValue(0) = (Main.colon)(1,n)
        #temp# = (Base.start)(SSAValue(0))
        6: 
        unless !((Base.done)(SSAValue(0),#temp#)) goto 15
        SSAValue(1) = (Base.next)(SSAValue(0),#temp#)
        i = (Core.getfield)(SSAValue(1),1)
        #temp# = (Core.getfield)(SSAValue(1),2) # line 4:
        x = x + (Main.randn)()
        13: 
        goto 6
        15:  # line 6:
        return x
    end)

In [14]:
@code_warntype loop(10)

Variables:
  #self#::#loop
  n::Int64
  x@_3::Float64
  #temp#@_4::Int64
  i::Int64
  r::UInt64
  rabs::Int64
  idx::Int64
  x@_9::Float64
  #temp#@_10::Union{Bool,Int64}
  #temp#@_11::Float64
  #temp#@_12::Float64

Body:
  begin 
      x@_3::Float64 = 0.0 # line 3:
      SSAValue(7) = (Base.select_value)((Base.sle_int)(1,n::Int64)::Bool,n::Int64,(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64
      #temp#@_4::Int64 = 1
      5: 
      unless (Base.box)(Base.Bool,(Base.not_int)((#temp#@_4::Int64 === (Base.box)(Int64,(Base.add_int)(SSAValue(7),1)))::Bool)) goto 97
      SSAValue(8) = #temp#@_4::Int64
      SSAValue(9) = (Base.box)(Int64,(Base.add_int)(#temp#@_4::Int64,1))
      i::Int64 = SSAValue(8)
      #temp#@_4::Int64 = SSAValue(9) # line 4:
      $(Expr(:inbounds, false))
      # meta: location random.jl randn 1129
      $(Expr(:inbounds, false))
      # meta: location random.jl randn 1129
      $(Expr(:inbounds, true)) # line 1130:
      $(Expr(:inbounds, false))
      # meta: loca

In [15]:
@code_llvm loop(10)


define double @julia_loop_71993(i64) #0 {
top:
  %1 = call %jl_value_t*** @jl_get_ptls_states() #1
  %2 = alloca [6 x %jl_value_t*], align 8
  %.sub = getelementptr inbounds [6 x %jl_value_t*], [6 x %jl_value_t*]* %2, i64 0, i64 0
  %3 = getelementptr [6 x %jl_value_t*], [6 x %jl_value_t*]* %2, i64 0, i64 2
  %4 = bitcast [6 x %jl_value_t*]* %2 to i64*
  %5 = bitcast %jl_value_t** %3 to i8*
  call void @llvm.memset.p0i8.i64(i8* %5, i8 0, i64 32, i32 8, i1 false)
  store i64 8, i64* %4, align 8
  %6 = bitcast %jl_value_t*** %1 to i64*
  %7 = load i64, i64* %6, align 8
  %8 = getelementptr [6 x %jl_value_t*], [6 x %jl_value_t*]* %2, i64 0, i64 1
  %9 = bitcast %jl_value_t** %8 to i64*
  store i64 %7, i64* %9, align 8
  store %jl_value_t** %.sub, %jl_value_t*** %1, align 8
  %10 = icmp slt i64 %0, 1
  br i1 %10, label %L11, label %if.lr.ph

if.lr.ph:                                         ; preds = %top
  %11 = getelementptr [6 x %jl_value_t*], [6 x %jl_value_t*]* %2, i64 0, i64 3
  %12

In [16]:
@code_native loop(10)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[11]
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	subq	$72, %rsp
	movq	%rdi, %rbx
	movabsq	$jl_get_ptls_states_fast, %rax
	callq	*%rax
	movq	%rax, -104(%rbp)
	movq	$0, -48(%rbp)
	movq	$0, -56(%rbp)
	movq	$0, -64(%rbp)
	movq	$0, -72(%rbp)
	movq	$8, -88(%rbp)
	movq	(%rax), %rcx
	movq	%rcx, -80(%rbp)
	leaq	-88(%rbp), %rcx
	movq	%rcx, (%rax)
	xorpd	%xmm0, %xmm0
Source line: 3
	testq	%rbx, %rbx
	jle	L330
	xorpd	%xmm0, %xmm0
	movabsq	$4616366056, %r15       ## imm = 0x1132827E8
	movabsq	$140429554550336, %r12  ## imm = 0x7FB84DB70640
Source line: 1131
	movabsq	$2251799813685247, %r14 ## imm = 0x7FFFFFFFFFFFF
Source line: 1135
	movabsq	$randn_unlikely, %r13
	nopw	%cs:(%rax,%rax)
L160:
	movsd	%xmm0, -96(%rbp)
Source line: 111
	movq	(%r15), %rax
	cmpq	$382, %rax              ## imm = 0x17E
	jne	L224
Source line: 107
	movq	-16(%r15), %rdi
	movq	%rdi, -72(%rbp)
	movq	-8(%r15), %rax
	movq	%ra